In [177]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as lr
import matplotlib.pyplot as plt

# Reading and cleaning data

In [53]:
# Reading train file:
df = pd.read_csv("train.csv")

# Transforming dates to months since 8/1/2019:
time = []

for date in df['first_day_of_month']:
	year, month, day = date.split('-')
	time.append((int(month) - 8 + 12*(int(year) - 2019)))

# Calculating number of months and number of different counties:
n_months = max(time) + 1
n_counties = int(len(time)/n_months)

# Defining the predictor variable X and response variable Y:
X = np.zeros((n_months,1))
for n in range(n_months):
	X[n] = [n+1]


# Cleaning the data -- remove counties with missing information
Yraw = np.zeros((n_counties, n_months))

for c in range(n_counties):
	Yraw[c,:] = df['microbusiness_density'][n_months*c:n_months*(c+1)]

Y = []
deleted_indices = []
for c in range(n_counties):
	if 0 in Yraw[c,:]:
		deleted_indices.append(c)
		pass
	else:
		Y.append(Yraw[c,:])

n_counties = len(Y)
Y = np.array(Y)

In [73]:
# Fitting Y to a linear regression model on X:
coeffs_m1 = np.zeros((n_counties,2))
coeffs_m2 = np.zeros((n_counties,2))
coeffs_m3 = np.zeros((n_counties,2))


# Model Baseline 1: Linear Regression over all months

In [74]:
for n in range(n_counties):
	reg = lr().fit(X, Y[n,:])
	coeffs_m1[n,0] = reg.coef_
	coeffs_m1[n,1] = reg.intercept_

# Model Baseline 2: Linear Regression over last 12 months

In [75]:
N2 = 12
for n in range(n_counties):
		reg = lr().fit(X[-N2:], Y[n,-N2:])
		coeffs_m2[n,0] = reg.coef_
		coeffs_m2[n,1] = reg.intercept_



# Model 3: Linear regression over selected data

In [76]:
n_min = 6
max_dev = 3
Nmonth = n_months*np.ones(n_counties, dtype = np.int8)

for n in range(n_counties):
	for N in range(n_min,n_months):
		reg = lr().fit(X[-N:], Y[n,-N:])
		coeffs_m3[n,0] = reg.coef_
		coeffs_m3[n,1] = reg.intercept_
		residuals = (Y[n,-N:].reshape(N,1)- X[-N:]*coeffs_m3[n,0] - coeffs_m3[n,1])
		std = np.std(residuals)
		new_residual = abs(Y[n,-N-1]- X[-N-1]*coeffs_m3[n,0] - coeffs_m3[n,1])
		if new_residual > max_dev*std:
			Nmonth[n] = int(N)
			break
            

# Computing standard error of the train data for each model:

In [161]:
# Computing the standard errors of prediction for each County:
se1 = []
se2 = []
se3 = []

for n in range(n_counties):
	Y_pred_m1 = X*coeffs_m1[n,0] + coeffs_m1[n,1]
	Y_pred_m2 = X[-N2:]*coeffs_m2[n,0] + coeffs_m2[n,1]
	Y_pred_m3 = X[-Nmonth[n]:]*coeffs_m3[n,0] + coeffs_m3[n,1]
	if 0 in Y[n,:]:
		pass
	else:
		se1.append(np.sum((abs(Y[n,:]-Y_pred_m1.T))/Y[n,:])/n_months)
		se2.append(np.sum((abs(Y[n,-N2:]-Y_pred_m2.T))/Y[n,-N2:])/N2)
		se3.append(np.sum((abs(Y[n,-Nmonth[n]:]-Y_pred_m3.T))/Y[n,-Nmonth[n]:])/Nmonth[n])


0.04552844437085259 0.015360244406760586 0.011176985013977223
0.026407058199239827 0.009153593064664432 0.006650828788040583
3.5392394454307046 1.6471254346994728 1.943835687516089


## Output 1: mean errors for train data

In [180]:
        
print(np.mean(se1),np.mean(se2),np.mean(se3))
print(np.median(se1),np.median(se2),np.median(se3))
print(np.max(se1),np.max(se2),np.max(se3))

0.04552844437085259 0.015360244406760586 0.011176985013977223
0.026407058199239827 0.009153593064664432 0.006650828788040583
3.5392394454307046 1.6471254346994728 1.943835687516089


# Evaluating error in test data:

In [80]:
# Reading the test file:
df2 = pd.read_csv("revealed_test.csv")

# Computing X_test and Y_test:
n_months_test = 2

X_test = []

for date in df2['first_day_of_month'][0:n_months_test]:
	year, month, day = date.split('-')
	time = ((int(month) - 8 + 12*(int(year) - 2019)))
	X_test.append([time])

X_test = np.array(X_test)
Y_test = []

for c in range(n_counties+len(deleted_indices)):
	if c in deleted_indices:
		pass
	else:
		Y_test.append(df2['microbusiness_density'][n_months_test*c:n_months_test*(c+1)])
    
Y_test = np.array(Y_test)

Y_test.shape

se_test1 = []
se_test2 = []
se_test3 = []

for n in range(n_counties):
	Y_pred_test_m1 = X_test*coeffs_m1[n,0] + coeffs_m1[n,1]
	Y_pred_test_m2 = X_test*coeffs_m2[n,0] + coeffs_m2[n,1]
	Y_pred_test_m3 = X_test*coeffs_m3[n,0] + coeffs_m3[n,1]
	se_test1.append(np.sum((abs(Y_test[n,:]-Y_pred_test_m1.T))/Y_test[n,:])/n_months_test)
	se_test2.append(np.sum((abs(Y_test[n,:]-Y_pred_test_m2.T))/Y_test[n,:])/n_months_test)
	se_test3.append(np.sum((abs(Y_test[n,:]-Y_pred_test_m3.T))/Y_test[n,:])/n_months_test)

## Output 2: mean errors for test data

In [178]:
print(np.mean(se_test1),np.mean(se_test2),np.mean(se_test3))
print(np.median(se_test1),np.median(se_test2),np.median(se_test3))
print(np.max(se_test1),np.max(se_test2),np.max(se_test3))

0.06708537412222045 0.027611896321677393 0.027614518780614418
0.034589751906966504 0.012638897013648296 0.011659280627290443
14.446930749306366 3.0452154390544885 5.552370074209941


# Analysis - tuning lambda


In [157]:
n_min = 6
max_devs = [1,2,3,4,5,6,7,8,9,10]
Nmonth2 = n_months*np.ones(n_counties, dtype = np.int8)
coeffs_m32 = np.zeros((n_counties,2))
se_test32_mean = []
se3_mean = []


for max_dev in max_devs:
    se_test32 = []
    se3 = []

    for n in range(n_counties):
        for N in range(n_min,n_months):
            reg = lr().fit(X[-N:], Y[n,-N:])
            coeffs_m32[n,0] = reg.coef_
            coeffs_m32[n,1] = reg.intercept_
            residuals = (Y[n,-N:].reshape(N,1)- X[-N:]*coeffs_m32[n,0] - coeffs_m32[n,1])
            std = np.std(residuals)
            new_residual = abs(Y[n,-N-1]- X[-N-1]*coeffs_m32[n,0] - coeffs_m32[n,1])
            if new_residual > max_dev*std:
                Nmonth2[n] = int(N)
                break

    # Computing the standard errors of prediction for each County:
    

    for n in range(n_counties):
        Y_pred_m32 = X[-Nmonth2[n]:]*coeffs_m32[n,0] + coeffs_m32[n,1]
        if 0 in Y[n,:]:
            pass
        else:
            se3.append(np.sum((abs(Y[n,-Nmonth2[n]:]-Y_pred_m32.T))/Y[n,-Nmonth2[n]:])/Nmonth2[n])

    

    for n in range(n_counties):
        Y_pred_test_m32 = X_test*coeffs_m32[n,0] + coeffs_m32[n,1]
        se_test32.append(np.sum((abs(Y_test[n,:]-Y_pred_test_m32.T))/Y_test[n,:])/n_months_test)

    se3_mean.append(np.mean(se3))
    se_test32_mean.append(np.mean(se_test32))



[0.010370775729483212, 0.010261081958558866, 0.012157803240149156, 0.015875345103973745, 0.01979046549261085, 0.023059862502616102, 0.025188426659647725, 0.026588029298505025, 0.02786818869126414, 0.028921298595061744] [0.02271675420750567, 0.02510591288223175, 0.027614518780614418, 0.034734574873631914, 0.03935437808924475, 0.04352895826771346, 0.04625518175273745, 0.0478423225396263, 0.04933933508521539, 0.05064938930455502]


## Output 3: mean errors under different lambdas

In [179]:
print(se3_mean, se_test32_mean)

[0.010370775729483212, 0.010261081958558866, 0.012157803240149156, 0.015875345103973745, 0.01979046549261085, 0.023059862502616102, 0.025188426659647725, 0.026588029298505025, 0.02786818869126414, 0.028921298595061744] [0.02271675420750567, 0.02510591288223175, 0.027614518780614418, 0.034734574873631914, 0.03935437808924475, 0.04352895826771346, 0.04625518175273745, 0.0478423225396263, 0.04933933508521539, 0.05064938930455502]
